<a href="https://colab.research.google.com/github/baichlseder/voila/blob/main/voila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install fastai --upgrade -q

In [23]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [ ]:
# https://drive.google.com/file/d/1-OujMPsJ5fPGSaCkoHQiJKiemcMyzOyK/view?usp=sharing

In [24]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
      params = { 'id' : id, 'confirm' : token }
      response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)
    
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [25]:
file_id = '1-OujMPsJ5fPGSaCkoHQiJKiemcMyzOyK'
destination = './handwritting_classifier.pkl'
download_file_from_google_drive(file_id, destination)

## Classifier zu Nummern
Es werden alle Zahlen von 0 bis 9 erkannt:
  * 0
  * 1
  * 2
  * 3
  * 4
  * 5
  * 6
  * 7
  * 8
  * 9

In [26]:
path = Path()
inference = load_learner('/content/handwritting_classifier.pkl', cpu=true)

In [27]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl : display(img.to_thumb(128,128))
  pred, pred_idx, probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your number!'),
              btn_upload, btn_run, out_pl, lbl_pred])